## Prepare Data + Model

In [1]:
# !cat examples/data/text_forward.txt

In [2]:
# !ls -al ./outputs/en.1-percent.elmo-bert-causal-fixr2l

In [3]:
import torch
from newlm.lm.elmo.modeling_elmo.elmo_head import ELMOBertLMHeadModel
from newlm.lm.elmo.lm_builder import ELMOLMBuilder
from transformers import BertConfig

#### Model

In [4]:
model = ELMOBertLMHeadModel.from_pretrained(
    "./outputs/en.100-percent.elmo-bert-causal.40k"
) # use pre-trained model

In [5]:
model.eval()
print("Model in eval mode for consistency")

Model in eval mode for consistency


#### Data

In [6]:
%%capture

from newlm.utils.file_util import read_from_yaml
config_file = read_from_yaml('examples/configs_gcloud/run-100-percent.elmo-bert-causal.yaml')

# lm builder (helper)
elmo_lm_builder = ELMOLMBuilder(
    model_config = config_file['lm']['model']['config'],
    tokenizer="./outputs/en.100-percent.elmo-bert-causal.40k", # use pre-trained tokenizer
    model_type="bert-causal-elmo",
    max_len=128
)

# dataset-forward
train_path = "./examples/data/text_forward-small.txt"
ds_f = elmo_lm_builder._get_dataset(train_path)

2021-11-22 19:46:05.850 | INFO     | newlm.lm.elmo.lm_builder:_get_dataset:142 - Constructing roBERTa style dataset


In [7]:
# trainer (helper)
from transformers import TrainingArguments, Trainer
args = TrainingArguments(output_dir="tmpout",**config_file['lm']['hf_trainer']['args'])

# dataloader-forward
trainer = Trainer(model=model, args=args, data_collator=elmo_lm_builder.data_collator, train_dataset=ds_f,)
dl_f = trainer.get_train_dataloader() # Data Loader-forward

max_steps is given, it will override any value given in num_train_epochs


In [8]:
batch_f = next(iter(dl_f))
batch_f['input_ids'].shape

torch.Size([1, 123])

In [9]:
model.eval()
print("Model in eval mode for consistency")

Model in eval mode for consistency


## Sanity Check

In [10]:
# batch_f

In [11]:
import torch

def reverse_batch(batch_f):
    # reverse input
    batch_f_input = torch.clone(batch_f['input_ids'])
    batch_f_rev_input = torch.cat(
        (
            batch_f_input[0][0:1],
            torch.flip(batch_f_input[0][1:-1], [0]),
            batch_f_input[0][-1:]
        )
    )
    batch_f_rev_input = batch_f_rev_input.reshape(1,-1)

    # reverse labels
    batch_f_rev_labels = torch.clone(batch_f_rev_input)
    
    # batch_rev
    batch_rev = batch_f.copy()
    batch_rev['input_ids'] = batch_f_rev_input
    batch_rev['labels'] = batch_f_rev_labels
    
    return batch_rev

In [12]:
import pandas as pd

def pandas_check(batch_f, batch_rev):
    tokens_f = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_f['input_ids'][0])
    tokens_f_rev = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_rev['input_ids'][0])
    return pd.DataFrame({"forward": tokens_f, "reverse": tokens_f_rev})

#### Normal vs Reverse

In [13]:
batch_rev = reverse_batch(batch_f)

In [14]:
pandas_check(batch_f, batch_rev)

,forward,reverse
0,[CLS],[CLS]
1,[UNK],.
2,is,Jews
3,located,among
4,in,lived
...,...,...
118,lived,in
119,among,located
120,Jews,is
121,.,[UNK]


In [15]:
batch_f['input_ids'].shape, batch_rev['input_ids'].shape

(torch.Size([1, 123]), torch.Size([1, 123]))

In [16]:
res = model(**batch_f) # forward

l2r_loss tensor(3.9125, grad_fn=<NllLossBackward>)
r2l_loss tensor(3.8753, grad_fn=<NllLossBackward>)


In [17]:
res = model(**batch_rev) # reverse

l2r_loss tensor(8.8075, grad_fn=<NllLossBackward>)
r2l_loss tensor(8.8464, grad_fn=<NllLossBackward>)


#### Random String

In [18]:
batch_f['input_ids'].shape

torch.Size([1, 123])

In [19]:
# shuffle data
batch_f_input = batch_f['input_ids']
batch_shuffle_input = torch.cat(
    (
        batch_f_input[0][0:1],
        torch.randint(
            low=5, # 0-4 > ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']
            high=29999,
            size=(121,), # modified based on seqlen!
            dtype=torch.long
        ),
        batch_f_input[0][-1:]
    )
)
batch_shuffle_input = batch_shuffle_input.reshape(1,-1)
# labels
batch_shuffle_labels = torch.clone(batch_shuffle_input)    
# batch_shuffle
batch_shuffle = batch_f.copy()
batch_shuffle['input_ids'] = batch_shuffle_input
batch_shuffle['labels'] = batch_shuffle_labels

In [20]:
batch_shuffle_rev = reverse_batch(batch_shuffle)

##### Trial-1

In [21]:
pandas_check(batch_shuffle, batch_shuffle_rev)

,forward,reverse
0,[CLS],[CLS]
1,##wich,battery
2,desk,Ner
3,Collection,ought
4,fisher,Mug
...,...,...
118,Mug,fisher
119,ought,Collection
120,Ner,desk
121,battery,##wich


In [22]:
res = model(**batch_shuffle) # forward

l2r_loss tensor(13.4756, grad_fn=<NllLossBackward>)
r2l_loss tensor(13.2225, grad_fn=<NllLossBackward>)


In [23]:
res = model(**batch_shuffle_rev) # reverse

l2r_loss tensor(13.2165, grad_fn=<NllLossBackward>)
r2l_loss tensor(13.3072, grad_fn=<NllLossBackward>)


##### Trial-2

In [24]:
pandas_check(batch_shuffle, batch_shuffle_rev)

,forward,reverse
0,[CLS],[CLS]
1,##wich,battery
2,desk,Ner
3,Collection,ought
4,fisher,Mug
...,...,...
118,Mug,fisher
119,ought,Collection
120,Ner,desk
121,battery,##wich


In [25]:
res = model(**batch_shuffle) # forward

l2r_loss tensor(13.4756, grad_fn=<NllLossBackward>)
r2l_loss tensor(13.2225, grad_fn=<NllLossBackward>)


In [26]:
res = model(**batch_shuffle_rev) # reverse

l2r_loss tensor(13.2165, grad_fn=<NllLossBackward>)
r2l_loss tensor(13.3072, grad_fn=<NllLossBackward>)


##### Trial-3

In [27]:
pandas_check(batch_shuffle, batch_shuffle_rev)

,forward,reverse
0,[CLS],[CLS]
1,##wich,battery
2,desk,Ner
3,Collection,ought
4,fisher,Mug
...,...,...
118,Mug,fisher
119,ought,Collection
120,Ner,desk
121,battery,##wich


In [28]:
res = model(**batch_shuffle) # forward

l2r_loss tensor(13.4756, grad_fn=<NllLossBackward>)
r2l_loss tensor(13.2225, grad_fn=<NllLossBackward>)


In [29]:
res = model(**batch_shuffle_rev) # reverse

l2r_loss tensor(13.2165, grad_fn=<NllLossBackward>)
r2l_loss tensor(13.3072, grad_fn=<NllLossBackward>)
